In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from datasets import load_from_disk, Dataset, DatasetDict, load_dataset
from torch.nn.utils.rnn import pad_sequence
import copy

In [3]:
model_name = "sberbank-ai/mGPT"

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_size='right')

special_tokens_dict = {
    'bos_token': '<BOS>',
    'eos_token': '<|endoftext|>',
    'pad_token': '<PAD>'}

num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

In [5]:
class PoemDataset(Dataset):
    def __init__(self, dataset_dict, tokenizer, seq_length=1024):
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        self.dataset = dataset_dict

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

      data = self.dataset[idx]
      poet, poem = data.get("poet", ""), data.get("poem", "")
      lines = " ".join(poem)
      first_line = lines.partition('\n')[0]

      text = f"<BOS> {poet} : {first_line}\n<|endoftext|>"
      # print(f'i:{idx}, text= {text}')


      input_encoding = self.tokenizer.encode_plus(
                                 text,
                                 max_length=self.seq_length,
                                 padding='max_length',
                                 return_attention_mask=True,
                                 return_tensors='pt',
                                 truncation=True
                                )

      input_ids = input_encoding['input_ids'].flatten()
      attention_masks = input_encoding['attention_mask'].flatten()

      target_encoding = self.tokenizer.encode_plus(
                                 poem,
                                 max_length=self.seq_length,
                                 padding='max_length',
                                 return_attention_mask=True,
                                 return_tensors='pt',
                                 truncation=True
                                )
      target_ids = target_encoding['input_ids'].flatten()


      return {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'target_ids': target_ids
        }


In [7]:
dataset_dict = load_from_disk('turkish_poems_cleaned')
#dataset_dict = load_dataset("beratcmn/instruction-turkish-poems")
poemDataset = PoemDataset(dataset_dict, tokenizer)

In [8]:
a = dataset_dict[1]
a.get("poet", "")

'Ümit Yaşar Oğuzcan'

In [9]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

train_size, val_size = train_val_split(0.9, poemDataset)
train_dataset, val_dataset = random_split(poemDataset, [train_size, val_size])

In [10]:
train_dataloader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=1)

val_dataloader = DataLoader(val_dataset,
                            sampler=SequentialSampler(val_dataset),
                            batch_size=1)

In [11]:
model = GPT2LMHeadModel.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)
model.resize_token_embeddings(len(tokenizer))

model_org = copy.deepcopy(model)

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

In [ ]:
model = copy.deepcopy(model_org)

In [13]:
# Freeze transformer layers except the first and the last one. Do not freeze any layernorms
for n, p in model.named_parameters():
    if 'transformer.h' in n:
        layer_num = int(n.split('.')[2])
        if 'ln_' not in n:
            if layer_num > 10:
                p.requires_grad = True
                print('Un Freeze', n)
            else:
                p.requires_grad = False
                print('Freeze', n)

Freeze transformer.h.0.attn.c_attn.weight
Freeze transformer.h.0.attn.c_attn.bias
Freeze transformer.h.0.attn.c_proj.weight
Freeze transformer.h.0.attn.c_proj.bias
Freeze transformer.h.0.mlp.c_fc.weight
Freeze transformer.h.0.mlp.c_fc.bias
Freeze transformer.h.0.mlp.c_proj.weight
Freeze transformer.h.0.mlp.c_proj.bias
Freeze transformer.h.1.attn.c_attn.weight
Freeze transformer.h.1.attn.c_attn.bias
Freeze transformer.h.1.attn.c_proj.weight
Freeze transformer.h.1.attn.c_proj.bias
Freeze transformer.h.1.mlp.c_fc.weight
Freeze transformer.h.1.mlp.c_fc.bias
Freeze transformer.h.1.mlp.c_proj.weight
Freeze transformer.h.1.mlp.c_proj.bias
Freeze transformer.h.2.attn.c_attn.weight
Freeze transformer.h.2.attn.c_attn.bias
Freeze transformer.h.2.attn.c_proj.weight
Freeze transformer.h.2.attn.c_proj.bias
Freeze transformer.h.2.mlp.c_fc.weight
Freeze transformer.h.2.mlp.c_fc.bias
Freeze transformer.h.2.mlp.c_proj.weight
Freeze transformer.h.2.mlp.c_proj.bias
Freeze transformer.h.3.attn.c_attn.weigh

In [14]:
#model.cuda()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-7)

In [15]:
model.train()
for epoch in range(2):
    print('Epoch', epoch)

    train_losses = []
    print('Training...')

    progressbar = tqdm(train_dataloader)
    for batch in progressbar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(model.device)
        labels = batch['target_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        progressbar.set_description("Loss: %.3f" % np.mean(train_losses[-10:]))


    val_losses = []
    print('Validating...')


    progressbar = tqdm(val_dataloader)
    for batch in progressbar:

        input_ids = batch['input_ids'].to(model.device)
        labels = batch['target_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)

        with torch.no_grad():

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            val_losses.append(loss.item())

            progressbar.set_description("Loss: %.3f" % np.mean(val_losses[-10:]))


Epoch 0
Training...


  0%|          | 0/450 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Loss: 0.078: 100%|██████████| 450/450 [4:38:59<00:00, 37.20s/it]


Validating...


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Loss: 0.091: 100%|██████████| 50/50 [10:16<00:00, 12.34s/it]


Epoch 1
Training...


  0%|          | 0/450 [00:35<?, ?it/s]


KeyboardInterrupt: 

In [20]:
torch.save(model.state_dict(), '/content/drive/MyDrive/model2.pth')

In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_name)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/content/drive/MyDrive/model.pth'))

<All keys matched successfully>

In [16]:
eos_id = tokenizer.encode("<EOS>")

In [21]:
input_ids = tokenizer.encode("<BOS>Pir Sultan Abdal : Söyle anlat ona,\n", return_tensors = "pt").cuda()
input_ids = input_ids.to(model.device)
out = model.generate(
        input_ids,
        num_beams=5,
        num_return_sequences=1,
        max_length=100,
        eos_token_id=eos_id,
        do_sample=True,
        top_k=6,
        top_p=0.92,
        temperature=0.9,
        no_repeat_ngram_size=3)

In [22]:
for sample_output in out:
    print("{} ".format(tokenizer.decode(sample_output, skip_special_tokens=True)))

Pir Sultan Abdal : Söyle anlat ona,
Seni seviyorum, seni seviyoruz,
Seninle gurur duymak istiyoruz, seninle
Gurur duyuyoruz. Seni sevdiğimizi biliyoruz.
Biz seni sevdik, biz seni seveceğiz,
Bizi sevdiğin gibi seveceksin, bizi seveceksin.
 
